In [1]:
import os
import sys
sys.path.append(os.getcwd()+'/..')

from pdearena import utils
import torch
from pdearena.data.datamodule import PDEDataModule
from pdearena.lr_scheduler import LinearWarmupCosineAnnealingLR  # noqa: F401
from pdearena.models.pdemodel import PDEModel

from helper_functions.data import write_h5, read_h5_numpy
from helper_functions.general import touch_dir
from time import time
import numpy as np
import h5py
from tqdm import tqdm
import shutil
import _pickle as cPickle
import pickle 
import hdf5plugin


In [2]:
import sys
sys.path


['/mnt/c/Users/eid0112947/OneDrive - Eni/Documenti/RESEARCH/pdearena/pdedatagen',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python38.zip',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8/lib-dynload',
 '',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8/site-packages',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/ENI/Projects/Prove/sci_ML/pdearena',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/ENI/Projects/Prove/helper_functions',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/Documenti/RESEARCH/pdearena/pdedatagen/..',
 '/tmp/tmpfo0jihb_']

In [3]:
def hdf5_to_dict(file):
    d = dict()
    for key,val in file.items():
        if type(val) == h5py._hl.dataset.Dataset:
            d[key] = np.array(val)
        else:
            d[key] = hdf5_to_dict(val)
    return d

In [4]:
source_folder = '../pdearena_data/NavierStokes-2D-64x64/'
target_folder = '../pdearena_data/NavierStokes-2D-64x64-io_test/data/'

touch_dir(target_folder)

# seeds = [946191, 795933, 585798]
n_keep= 1
files_keep = [f for f in os.listdir(source_folder) if f.find('train')!=-1 & f.find('yaml')==-1][:n_keep]



for file in tqdm(
        files_keep, 
        total=len(files_keep), 
        miniters=len(files_keep)//5,
        leave=False):
    with h5py.File(os.path.join(source_folder,file), 'r') as hf_source:
        here = 1
        d = hdf5_to_dict(hf_source)
        with h5py.File(os.path.join(target_folder,file), "w") as h5f_target:
            # List all groups
            # print("Keys: %s" % h5f.keys())
            mode = next(iter(d.keys()))
            dataset = h5f_target.create_group(mode)
            for k,v in d[mode].items():
                print('processing {}'.format(k))
                chunks = (100,)
                if len(v.shape) > 1:
                    for i in v.shape[1:]:  
                        chunks=chunks+(i,)
                dataset.create_dataset(
                    k, 
                    data = v,
                    shape = v.shape,
                    # chunks = chunks
                )

file = target_folder +  files_keep[0]

n_test = 10
t1 = time()
for _ in range(n_test):
    read_h5_numpy(file)
t2 = time() - t1
print('time per step: {:0.5f}s'.format(t2/n_test))

  0%|          | 0/1 [00:00<?, ?it/s]

processing buo_y
processing dt
processing dx
processing dy
processing t
processing u
processing vx


processing vy
processing x
processing y


time per step: 0.39934s


In [12]:
# test h5py with other compressions

source_folder = '../pdearena_data/NavierStokes-2D-64x64/'
target_folder = '../pdearena_data/NavierStokes-2D-64x64-io_test/data/'

touch_dir(target_folder)

# seeds = [946191, 795933, 585798]
n_keep= 1
files_keep = [f for f in os.listdir(source_folder) if f.find('train')!=-1 & f.find('yaml')==-1][:n_keep]

compressor = hdf5plugin.LZ4()
file_save = files_keep[0][:-3] + '_lz4' + '.h5'

# compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)
# file_save = files_keep[0][:-3] + '_Blosc_lz4' + '.h5'

# compressor = hdf5plugin.Blosc(cname='blosclz', clevel = 1)
# file_save = files_keep[0][:-3] + '_Blosc_blosclz' + '.h5'

# compressor = hdf5plugin.Blosc(cname='snappy', clevel = 1)
# file_save = files_keep[0][:-3] + '_Blosc_snappy' + '.h5'

# compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)
# file_save = files_keep[0][:-3] + '_Blosc_lz4' + '.h5'

compressor = hdf5plugin.Blosc(cname='zstd', clevel = 1)
file_save = files_keep[0][:-3] + '_Blosc_zstd' + '.h5'

# compressor = hdf5plugin.Blosc2(cname='lz4', clevel = 1)
# file_save = files_keep[0][:-3] + '_Blosc2_lz4' + '.h5'

# compressor = hdf5plugin.Blosc2(cname='zstd', clevel = 1)
# file_save = files_keep[0][:-3] + '_Blosc2_zstd' + '.h5'

# compressor = hdf5plugin.Blosc2(cname='blosclz', clevel = 1)
# file_save = files_keep[0][:-3] + '_Blosc2_blosclz' + '.h5'

# compressor = hdf5plugin.BZip2()
# file_save = files_keep[0][:-3] + '_BZip2' + '.h5'

os.path.join(target_folder,file_save)
os.path.join(source_folder,file)


for file in tqdm(
        files_keep, 
        total=len(files_keep), 
        miniters=len(files_keep)//5,
        leave=False):
    with h5py.File(os.path.join(source_folder,file), 'r') as hf_source:
        here = 1
        d = hdf5_to_dict(hf_source)
        with h5py.File(os.path.join(target_folder,file_save), "w") as h5f_target:
            # List all groups
            # print("Keys: %s" % h5f.keys())
            mode = next(iter(d.keys()))
            dataset = h5f_target.create_group(mode)
            for k,v in d[mode].items():
                # print('processing {}'.format(k))
                chunks = (100,)
                if len(v.shape) > 1:
                    for i in v.shape[1:]:  
                        chunks=chunks+(i,)
                dataset.create_dataset(
                    k, 
                    data = v,
                    shape = v.shape,
                    **compressor
                )

file = target_folder +  files_keep[0]

n_test = 10
t1 = time()
for _ in range(n_test):
    read_h5_numpy(file)
t2 = time() - t1
print('time per step: {:0.5f}s'.format(t2/n_test))

time per step: 0.37154s


In [34]:
# test pickle

'''
filehandler = open(b"Fruits.obj","wb")
pickle.dump(banana,filehandler)

file = open("Fruits.obj",'rb')
object_file = pickle.load(file)
'''

# test h5py with other compressions
import hdf5plugin

source_folder = '../pdearena_data/NavierStokes-2D-64x64/'
target_folder = '../pdearena_data/NavierStokes-2D-64x64-io_test/data/'

touch_dir(target_folder)

n_keep= 1
files_keep = [f for f in os.listdir(source_folder) if f.find('train')!=-1 & f.find('yaml')==-1][:n_keep]

file
file_save = files_keep[0][:-3] + '.pkl'

os.path.join(target_folder,file_save)
os.path.join(source_folder,file)


for file in tqdm(
        files_keep, 
        total=len(files_keep), 
        miniters=len(files_keep)//5,
        leave=False):
    with h5py.File(os.path.join(source_folder,file), 'r') as hf_source:
        here = 1
        d = hdf5_to_dict(hf_source)
        filehandler = open(os.path.join(target_folder,file_save),"wb")
        cPickle.dump(d,filehandler,protocol=pickle.HIGHEST_PROTOCOL)


file = target_folder +  file_save
file_save
n_test = 10
t1 = time()
for _ in range(n_test):
    filehandler = open(file,'rb')
    object_file = cPickle.load(filehandler)
t2 = time() - t1
print('time per step: {:0.5f}s'.format(t2/n_test))


time per step: 0.31201s


In [36]:
# test parquet

'''
NOT WORKING
'''
# import pyarrow as pa
# import pyarrow.parquet as pq

# '''
# data_dict = {"name": ["Alice"], "age": [30], "city": ["New York"]}
# table = pa.Table.from_pydict(data_dict)
# pq.write_table(table, 'data.parquet')
# '''
# source_folder = '../pdearena_data/NavierStokes-2D-64x64/'
# target_folder = '../pdearena_data/NavierStokes-2D-64x64-io_test/data/'

# touch_dir(target_folder)

# n_keep= 1
# files_keep = [f for f in os.listdir(source_folder) if f.find('train')!=-1 & f.find('yaml')==-1][:n_keep]

# file
# file_save = files_keep[0][:-3] + '.parquet'

# os.path.join(target_folder,file_save)
# os.path.join(source_folder,file)


# for file in tqdm(
#         files_keep, 
#         total=len(files_keep), 
#         miniters=len(files_keep)//5,
#         leave=False):
#     with h5py.File(os.path.join(source_folder,file), 'r') as hf_source:
#         here = 1
#         d = hdf5_to_dict(hf_source)
#         table = pa.Table.from_pydict(d[next(iter(d.keys()))])
#         pq.write_table(table, os.path.join(target_folder,file_save))


# file = target_folder +  file_save
# file_save
# n_test = 10
# t1 = time()
# for _ in range(n_test):
#     filehandler = open(file,'rb')
#     object_file = cPickle.load(filehandler)
# t2 = time() - t1
# print('time per step: {:0.5f}s'.format(t2/n_test))


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
files = [
    '../pdearena_data/NavierStokes-2D-64x64-sample/data/NavierStokes2D_train_198010_0.50000_100.h5',
    '../pdearena_data/NavierStokes-2D-64x64/NavierStokes2D_train_126288_0.50000_100.h5',
    ] 

dir = '../pdearena_data/NavierStokes-2D-64x64-io_test/data/'
files.extend([dir + f for f in os.listdir(dir)])

n_test = 10
for file in files:
    t1 = time()
    for _ in range(n_test):
        if 'pkl' in file:
            filehandler = open(file,'rb')
            object_file = cPickle.load(filehandler)
        else:
            read_h5_numpy(file)
    t2 = time() - t1
    print('file {} - time per step: {:0.5f}s'.format(
        file.split('/')[-1],
        t2/n_test)
        )

file NavierStokes2D_train_198010_0.50000_100.h5 - time per step: 0.59909s
file NavierStokes2D_train_126288_0.50000_100.h5 - time per step: 0.33895s
file NavierStokes2D_train_126288_0.50000_100.h5 - time per step: 0.34188s
file NavierStokes2D_train_126288_0.50000_100.pkl - time per step: 0.31170s
file NavierStokes2D_train_126288_0.50000_100_Blosc2_blosclz.h5 - time per step: 0.40147s
file NavierStokes2D_train_126288_0.50000_100_Blosc2_lz4.h5 - time per step: 0.37916s
file NavierStokes2D_train_126288_0.50000_100_Blosc2_zstd.h5 - time per step: 0.44810s
file NavierStokes2D_train_126288_0.50000_100_Blosc_blosclz.h5 - time per step: 0.38021s
file NavierStokes2D_train_126288_0.50000_100_Blosc_lz4.h5 - time per step: 0.37658s
file NavierStokes2D_train_126288_0.50000_100_Blosc_snappy.h5 - time per step: 0.46755s
file NavierStokes2D_train_126288_0.50000_100_Blosc_zstd.h5 - time per step: 0.48643s
file NavierStokes2D_train_126288_0.50000_100_BZip2.h5 - time per step: 3.71461s
file NavierStokes2D

In [5]:
### Chosen the following saving strategies
'''
- hdf5plugin.Blosc(cname='lz4', clevel = 1)
- pickle
'''
compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)

input_folder = '../pdearena_data/NavierStokes-2D-64x64/'
target_folders = [
    '../pdearena_data/NavierStokes-2D-64x64_blosc_lz4/',
    '../pdearena_data/NavierStokes-2D-64x64_pkl/',
]
files = os.listdir(input_folder)


def convert_file(filein,fileout):
    with h5py.File(filein, 'r') as hf_source:
        d = hdf5_to_dict(hf_source)
        if 'pkl' in fileout:
            with open(fileout,"wb") as filehandler:
                cPickle.dump(d,filehandler,protocol=pickle.HIGHEST_PROTOCOL)
            # pass
        else:
            with h5py.File(fileout, "w") as h5f_target:
                mode = next(iter(d.keys()))
                dataset = h5f_target.create_group(mode)
                for k,v in d[mode].items():
                    # print('processing {}'.format(k))
                    dataset.create_dataset(
                        k, 
                        data = v,
                        shape = v.shape,
                        **compressor
                    )



for target_folder in target_folders:
    touch_dir(target_folder)
    
    for file in tqdm(files,total=len(files)):
        try:
            if 'yaml' not in file:
                fileout = file
                if 'pkl' in target_folder:
                    fileout = fileout[:-3] + '.pkl'
                convert_file(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
            else:
                fileout = file
                shutil.copyfile(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
        except:
            print(file,fileout)
        



100%|██████████| 312/312 [01:19<00:00,  3.95it/s] 


In [16]:
target_folders = '../pdearena_data/NavierStokes-2D-64x64_pkl/'
files = os.listdir(target_folders)
file = [x for x in files if 'valid' in x][0]
file = os.path.join(target_folders,file)

file = '/mnt/c/Users/eid0112947/OneDrive - Eni/Documenti/RESEARCH/pdearena/pdearena_data/NavierStokes-2D-64x64_pkl/NavierStokes2D_valid_61849_0.50000.pkl'
with open(file,'rb') as filehandler: 
    object_file = cPickle.load(filehandler)

data = object_file['valid']
num = data["u"].shape[0]

iter_start = 0
iter_end = num

for idx in range(iter_start, iter_end):
    u = torch.tensor(data["u"][idx])
    vx = torch.tensor(data["vx"][idx])
    vy = torch.tensor(data["vy"][idx])
    if "buo_y" in data and False:
        cond = torch.tensor(data["buo_y"][idx]).unsqueeze(0).float()
    else:
        cond = None

    v = torch.cat((vx[:, None], vy[:, None]), dim=1)

    if False:
        gridx = torch.linspace(0, 1, data["x"][idx].shape[0])
        gridy = torch.linspace(0, 1, data["y"][idx].shape[0])
        gridx = gridx.reshape(1,gridx.size(0),1,).repeat(1,1,gridy.size(0))
        gridy = gridy.reshape(1,1,gridy.size(0),).repeat(1,gridx.size(1),1,)
        grid = torch.cat((gridx[:, None], gridy[:, None]), dim=1)
    else:
        grid = None
    if idx >= 23:
        print(
            type(u.unsqueeze(1).float()), 
            type(v.float()), 
            type(cond), 
            type(grid)
        )

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'NoneType'> <class 'NoneType'>
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'NoneType'> <class 'NoneType'>
